# Guaranteeing Custom Pydantic Schema

It is becoming increasingly important that models can output a custom schema that is guaranteed to compile. This notebook presents a framework for generating arbitrary schema outputs based on Pydantic's `BaseModel`.

In [4]:
# Creating our Pydantic BaseModel class
from pydantic import BaseModel, Field

class SteakhouseOrder(BaseModel):
    """Inputs for create_steak_order function for a fictional steakhouse"""

    table_number: int = Field(description="What table the order will be going to")
    customer_name: str = Field(description="Name of the customer")
    cooked: str = Field(description="How long to cook the steak, i.e. Medium, Rare, Well-done")
    sides: str = Field(
        description="Any sides that they may have ordered"
    )
    additional_requests: str = Field(
        description="Any additional info that is helpful for the order"
    )

SteakhouseOrder.schema()

{'title': 'SteakhouseOrder',
 'description': 'Inputs for create_steak_order function for a fictional steakhouse',
 'type': 'object',
 'properties': {'table_number': {'title': 'Table Number',
   'description': 'What table the order will be going to',
   'type': 'integer'},
  'customer_name': {'title': 'Customer Name',
   'description': 'Name of the customer',
   'type': 'string'},
  'cooked': {'title': 'Cooked',
   'description': 'How long to cook the steak, i.e. Medium, Rare, Well-done',
   'type': 'string'},
  'sides': {'title': 'Sides',
   'description': 'Any sides that they may have ordered',
   'type': 'string'},
  'additional_requests': {'title': 'Additional Requests',
   'description': 'Any additional info that is helpful for the order',
   'type': 'string'}},
 'required': ['table_number',
  'customer_name',
  'cooked',
  'sides',
  'additional_requests']}

### Guidance Functions to Generate Values

Below, `gen_properties` is the function that will generate the value for each pydantic property using the `gen` function. `generate_with_schema` takes in a user prompt through `user_instructions` and then generates the properties using the user input and pydantic schema definition. 

In [49]:
import guidance
from guidance import models, gen

@guidance
def gen_properties(lm, props):

    first = True
    for key, value in props.get("properties").items():
        lm += f'''{"," if not first else ""}
        {{
            "title": "{value.get('title')}",
            "key": "{key}",
            "description": "{value.get('description')}",
            "type": "{value.get('type')}",
            "value": ''' + gen("prop_value", stop="\n")
        
        + f'''
        }}
        '''
        first = False
    
    return lm

@guidance
def generate_with_schema(
    lm,
    user_instructions,
    schema
):
    lm += f'''\

    User instructions: {user_instructions}

    Result JSON:
    {{
        "props": [{gen_properties(schema)}]
    }}'''
    return lm

In [ ]:
# Load our model
model = 'mistralai/Mistral-7B-v0.1'
device = 'cuda'
lm = models.Transformers(model, device=device)

In [56]:
generation = lm + generate_with_schema(
    user_instructions="Create an order for John at table 9 for a medium-rare steak with a side of greenbeans. He has an almond allergy so be careful.", 
    schema=SteakhouseOrder.schema()
)

### Extracting Generated JSON and Pydantic output

Now that we have generated the values for our `SteahouseOrder`, we can extract them by using the `json` library and parsing out the JSON keys and values. We can then create a new instance of our Pydantic class with these generated values.

In [55]:
import json

# Extract the output from our result JSON
def parse_lm_output(lm_out):
    gen_str = generation.__str__()

    json_string_indicator = "Result JSON:"
    json_start_index = gen_str.find(json_string_indicator) + len(json_string_indicator) + 1
    full_json = gen_str[json_start_index:]

    gen_json = json.loads(full_json)
    final_json = {}

    for prop in gen_json['props']:
        final_json[prop['key']] = prop['value']

    return final_json

final_order = parse_lm_output(generation)

# Resulting output in dictionary
print(final_order)

# Resulting output in our original class
print(SteakhouseOrder(**final_order))

{'table_number': 9, 'customer_name': 'John', 'cooked': 'Medium-rare', 'sides': 'Greenbeans', 'additional_requests': 'No almonds'}
table_number=9 customer_name='John' cooked='Medium-rare' sides='Greenbeans' additional_requests='No almonds'
